![](./resources/System_v1_demo_header.png)

This notebook contains a demo of the WorldCereal system v1 and all its functionalities.

# Content

- [1. Define region of interest](#1.-Define-a-region-of-interest)
- [2. Generate default Worldcereal products](#2.-Generate-default-WorldCereal-products)
- [3. Explore available reference data](#3.-Explore-available-reference-data)

<!-- - Contributing reference data to RDM
- Requesting reference data from RDM
- Launching point extractions for obtaining satellite input data 
- Launching catboost model training based on the extracted points
- Perform inference run with the newly trained model
    (and/or the default cropland model) -->

In [1]:
# First we import the necessary modules to run this notebook

import pandas as pd
import geopandas as gpd
import requests
from shapely.geometry import shape, Polygon

from worldcereal.utils.map import get_ui_map
from worldcereal.utils.refdata import _to_points

RDM_API = 'https://ewoc-rdm-api.iiasa.ac.at'

# 1. Define a region of interest

When running the code snippet below, an interactive map will be visualized.
Click the Rectangle button on the left hand side of the map to start drawing your region of interest.
When finished, execute the second cell to store the coordinates of your region of interest. 

In [2]:
m, dc = get_ui_map()
m

Map(center=[51.1872, 5.1154], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoo…

In [3]:
# retrieve bounding box from drawn rectangle
obj = dc.last_draw
if obj.get('geometry') is not None:
    poly = Polygon(shape(obj.get('geometry')))
    bbox = poly.bounds
else:
    raise ValueError('Please first draw a rectangle '
                     'on the map before proceeding.')
print(f'Your area of interest: {bbox}')

Your area of interest: (3.975677, 51.297564, 4.079361, 51.323747)


# 2. Generate default WorldCereal products

In this section we will generate a WorldCereal mapping product for our region of interest using a series of pre-trained models made available by the consortium. Aside from our region, we will need to specify the type of product to generate and the time frame (year or agricultural season) of interest.

In this example, we will generate a temporary crops product, followed by a crop type product for maize.

# 3. Explore available reference data

In-situ reference data used for training and validating WorldCereal classification models is hosted in the WorldCereal Reference Data Module (RDM). This module can be accessed through a dedicated user interface (https://ewoc-rdm-ui.iiasa.ac.at/map), but also through an API, which will be demonstrated here.

In this section we will explore the available reference datasets for our region of interest. In case you would like to test multiple regions of interest, execute the first two cells in [this section](#1.-Define-a-region-of-interest) of the notebook.

In [4]:
# Check full list of available collections
headers = {}
collectionResponse = requests.get(f'{RDM_API}/collections', headers=headers)
collections = collectionResponse.json()
col_ids = [x['collectionId'] for x in collections['items']]
print(f'Available collections: {col_ids}')

# Now we check which collections intersect with our AOI
bbox_str = f'Bbox={bbox[0]}&Bbox={bbox[1]}&Bbox={bbox[2]}&Bbox={bbox[3]}'
colSearchUrl = f'{RDM_API}/collections/search?{bbox_str}'
colSearchResponse = requests.get(colSearchUrl, headers=headers)
test = colSearchResponse.json()
print('The following collections intersect with your AOI:')
for i, col in enumerate(test):
    print()
    print(f'Collection {i+1}: {col["collectionId"]}')


Available collections: ['2017afoneacrefundmelpoint110', '2017ascawaprojectpoly111', '2017bellpisflanderspoly110', '2017bfajecamciradpoly111', '2017brajecamciradpoly111', '2017canaafccropinventorypoint110', '2017cmrcgiargardianpoint110', '2017lbnfaowapor1poly111', '2017lbnfaowapor2poly111', '2017mdgjecamciradpoly111']
The following collections intersect with your AOI:

Collection 1: 2017bellpisflanderspoly110

Collection 2: 2018eulucas2018point110

Collection 3: 2019beflandersfullpoly110

Collection 4: 2020nleurocroppoly110


In [ ]:
# Now that our reference data has been added,
# we launch a request to extract all reference points and polygons
# intersecting our region of interest.
# --> this should return the selected points/polygons as a geojson file/object
# (one per collection).

# To combine the data into one database, we immediately convert
# the polygons to points (by taking the centroid).

max_items = 2000

dfs = []
for col in cols_aoi:
    itemSearchCollectionId = col['collectionId']
    print(
        f'Extracting reference data from collection {itemSearchCollectionId}')
    itemSearchUrl = f'{RDM_API}/collections/{itemSearchCollectionId}/items?{bbox_str}&MaxResultCount={max_items}'
    itemSearchResponse = requests.get(itemSearchUrl, headers=headers)
    df = gpd.GeoDataFrame.from_features(
        itemSearchResponse.json(), crs='EPSG:4326')
    dfs.append(_to_points(df))

gdf = pd.concat(dfs, ignore_index=True)
print(f'Got a total of {len(gdf)} reference points')